# HW7.

by Handan Cetin | USCID: 6074572947 | github: handancetin

*Note*: I am aware the submission is 1 day late. I request to use late day policy to not get 10% penalty for my late submission. Thank you!

In [1]:
import os
import csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import  train_test_split, GridSearchCV
from sklearn.svm import SVC, LinearSVC
import sklearn.metrics as skM
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore') # for plots

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### 1.(a) Download the Anuran Calls (MFCCs) Data Set 
from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. 

Choose 70% of the data randomly as the training set

In [2]:
data = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
data = data.drop(columns = 'RecordID') # not informative
trainingSet, testingSet = train_test_split(data, train_size = 0.7, random_state = 1234)
trainingSet

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species
7095,1.0,-0.030661,0.709743,0.596995,0.016808,-0.032207,-0.017982,0.197515,0.180047,-0.147032,...,0.032437,0.088914,-0.005265,-0.070591,-0.001769,0.032462,0.050408,Hylidae,Scinax,ScinaxRuber
2829,1.0,0.450778,0.179596,0.536880,0.237089,-0.002735,-0.197700,0.040483,0.217061,-0.014972,...,-0.053893,0.166933,0.122329,-0.016720,-0.139801,0.016193,0.202066,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
6106,1.0,0.152550,0.350817,0.313831,0.165520,0.193691,0.124401,-0.130999,-0.039587,0.080869,...,0.067592,0.121362,0.033514,-0.004452,-0.050959,-0.073919,0.022301,Hylidae,Hypsiboas,HypsiboasCordobae
2237,1.0,0.234792,0.094121,0.471766,0.215810,-0.012293,-0.166575,0.027065,0.212956,-0.006380,...,0.032130,0.140456,0.006159,-0.142197,-0.125544,0.109423,0.209485,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1410,1.0,0.309895,0.294236,0.534573,0.169352,-0.009950,-0.176269,0.023710,0.286761,-0.066856,...,0.165935,0.245725,-0.010285,-0.137441,-0.100609,0.098605,0.197269,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,1.0,0.488300,0.690444,0.407224,-0.237701,0.006724,0.363194,-0.019622,-0.269613,0.235703,...,-0.122279,0.130141,0.079810,-0.108020,-0.013932,0.100946,0.013201,Leptodactylidae,Adenomera,AdenomeraAndre
3276,1.0,0.113519,-0.027441,0.394245,0.216591,0.076355,-0.080230,-0.006437,0.244941,0.168226,...,0.011259,0.293682,0.139877,-0.085210,-0.194582,-0.072365,0.137760,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1318,1.0,0.224324,0.261916,0.545881,0.058547,-0.033595,-0.119354,0.018393,0.254668,-0.020042,...,0.247929,0.173373,-0.074003,-0.063467,-0.026456,0.193278,0.212049,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
723,1.0,0.184451,-0.126894,0.194306,0.309350,0.275966,0.143157,-0.188311,-0.276036,0.002468,...,-0.236602,0.012743,0.167952,0.045986,-0.071974,-0.032195,0.004737,Dendrobatidae,Ameerega,Ameeregatrivittata


In [3]:
testingSet

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species
358,1.00000,0.539754,0.561840,0.485820,-0.146178,0.004778,0.339329,0.059313,-0.189063,0.128726,...,-0.074694,0.100696,0.116134,-0.030100,0.015457,0.112999,0.006357,Leptodactylidae,Adenomera,AdenomeraAndre
5506,1.00000,0.554986,0.472957,0.183668,0.022795,0.226007,0.046965,-0.105113,0.062718,0.093281,...,0.058479,-0.024091,-0.079565,0.025839,0.042072,0.010364,0.031573,Hylidae,Hypsiboas,HypsiboasCordobae
1818,1.00000,0.191909,0.142890,0.549125,0.164868,-0.031248,-0.209585,0.011656,0.279329,0.116283,...,-0.009474,0.216348,0.091516,-0.058038,-0.137571,0.012843,0.192080,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2816,1.00000,0.334923,0.250346,0.530691,0.213694,0.068699,-0.017036,0.066742,0.254933,0.053529,...,-0.001755,0.219171,0.047826,-0.042618,-0.116364,-0.086343,0.164200,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5376,0.93051,0.645139,1.000000,0.395311,-0.344746,0.298312,0.170154,-0.048689,0.138446,-0.116820,...,0.087468,-0.161493,-0.053657,0.130147,0.133926,-0.019497,-0.036147,Hylidae,Hypsiboas,HypsiboasCinerascens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6821,1.00000,0.279780,0.751817,0.310120,-0.262166,0.369200,0.308781,-0.238199,0.031779,0.181470,...,-0.071294,0.073567,0.004491,-0.046089,0.041174,0.085145,-0.009626,Leptodactylidae,Leptodactylus,LeptodactylusFuscus
1376,1.00000,0.343635,0.324422,0.472852,0.107253,0.009700,-0.117071,0.074943,0.271154,-0.024176,...,0.215127,0.172492,-0.043412,-0.129855,-0.076402,0.199216,0.194106,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2398,1.00000,0.170717,0.144414,0.549158,0.211581,-0.024628,-0.114756,0.123335,0.253060,-0.055609,...,0.107082,0.134105,0.011404,-0.058421,-0.061842,0.097548,0.120786,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2929,1.00000,0.232900,0.212858,0.527615,0.148104,0.061082,-0.046121,0.049468,0.325431,0.072443,...,0.124899,0.163374,-0.065837,-0.117566,-0.065872,0.086441,0.182685,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


### 1.(b) Each instance has three labels: Families, Genus, and Species. 
Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem.
One of the most important approaches to multi-label classification is to train a
classifier for each label (binary relevance).

In [4]:
labels = ['Family', 'Genus', 'Species']

trainingTarget = trainingSet[labels]
trainingFeatures = trainingSet.drop(columns = labels)

testingTarget = testingSet[labels]
testingFeatures = testingSet.drop(columns = labels)

#### 1.(a)i. Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem

1. Exact match ratio, also called sample-based accuracy, is the percentage of instances where the predicted labels exactly match the true labels. Mathematically, it is calculated as:

    EMR = (number of correct predictions / total number of instances)

2. Hamming score and hamming loss measures (they sum up to 1), is the average fraction of incorrect labels per instance, and it is calculated as:

    HL = (number of incorrect labels / total number of unique labels across all instances)
   
   HS = 1 - HL

#### 1.(a)ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. 
Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation.

In [6]:
# Perform SVM for each label (https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)
parameters = [{     'C': [0.001, 0.01, 0.1, 1, 10, 100],
                'gamma': [10, 1, 0.1, 0.01]}]
for label in labels:
    gridSearch = GridSearchCV(SVC(kernel = 'rbf'), parameters, cv = 10)
    gridSearch.fit(trainingFeatures, trainingTarget[[label]].values.ravel())
    testingPredicted = gridSearch.predict(testingFeatures)    
    print('\n--- Predictions for label:', label, '---')
    print('Best parameter found as:', gridSearch.best_params_)
    print('Exact Match Ratio =', np.round(skM.accuracy_score(testingTarget[[label]], testingPredicted), 4))
    print('Hamming Loss =', np.round(skM.hamming_loss(testingTarget[[label]], testingPredicted), 4))

--- Predictions for label: Family ---
Best parameter found as: {'C': 10, 'gamma': 1}
Exact Match Ratio = 0.9903
Hamming Loss = 0.0097
--- Predictions for label: Genus ---
Best parameter found as: {'C': 10, 'gamma': 1}
Exact Match Ratio = 0.988
Hamming Loss = 0.012
--- Predictions for label: Species ---
Best parameter found as: {'C': 10, 'gamma': 1}
Exact Match Ratio = 0.9884
Hamming Loss = 0.0116


#### 1.(a)iii. Repeat 1(b)ii with L1-penalized SVMs.
Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.

In [7]:
# Standardize the data
stdScaler = StandardScaler().fit(trainingFeatures)
trainingFeaturesTransformed = stdScaler.transform(trainingFeatures)
testingFeaturesTransformed = stdScaler.transform(testingFeatures)

# Perform SVM for each label
parameters = [{ 'C': [0.001, 0.01, 0.1, 1, 10, 100]}]
for label in labels:
    gridSearch = GridSearchCV(LinearSVC(penalty = 'l1', dual = False), parameters, cv = 10)
    gridSearch.fit(trainingFeaturesTransformed, trainingTarget[[label]].values.ravel())
    testingPredicted = gridSearch.predict(testingFeaturesTransformed)    
    print('\n---  Predictions for label:', label, '---')
    print('Best parameter found as:', gridSearch.best_params_)
    print('Exact Match Ratio =', np.round(skM.accuracy_score(testingTarget[[label]], testingPredicted), 4))
    print('Hamming Loss = ', np.round(skM.hamming_loss(testingTarget[[label]], testingPredicted), 4))


---  Predictions for label: Family ---
Best parameter found as: {'C': 1}
Exact Match Ratio = 0.9328
Hamming Loss =  0.0672

---  Predictions for label: Genus ---
Best parameter found as: {'C': 10}
Exact Match Ratio = 0.944
Hamming Loss =  0.056

---  Predictions for label: Species ---
Best parameter found as: {'C': 10}
Exact Match Ratio = 0.9565
Hamming Loss =  0.0435


#### 1.(a)iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [8]:
# Perform SMOTE (on standardized data) for each label
parameters = [{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
for label in labels:
    trainingFeaturesRes, trainingTargetRes = SMOTE(random_state = 123).fit_resample(trainingFeaturesTransformed, trainingTarget[[label]])
    gridSearch = GridSearchCV(LinearSVC(penalty = 'l1', dual = False, max_iter = 10000), parameters, cv = 10)
    gridSearch.fit(trainingFeaturesRes, trainingTargetRes[[label]].values.ravel())
    testingPredicted = gridSearch.predict(testingFeaturesTransformed)    
    print('\n---  Predictions for label:', label, '---')
    print('Best parameter found as:', gridSearch.best_params_)
    print('Exact Match Ratio =', np.round(skM.accuracy_score(testingTarget[[label]], testingPredicted), 4))
    print('Hamming Loss = ', np.round(skM.hamming_loss(testingTarget[[label]], testingPredicted), 4))


---  Predictions for label: Family ---
Best parameter found as: {'C': 10}
Exact Match Ratio = 0.9259
Hamming Loss =  0.0741

---  Predictions for label: Genus ---
Best parameter found as: {'C': 100}
Exact Match Ratio = 0.9092
Hamming Loss =  0.0908

---  Predictions for label: Species ---
Best parameter found as: {'C': 100}
Exact Match Ratio = 0.95
Hamming Loss =  0.05


By comparing the exact match ratios and hamming loss measures, it can be seen that gaussian kernel performs the best, followed by L1-penalized method, and SMOTE performs the worst for all labels.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
Monte-Carlo Simulation: Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate.

### 2.(a, b, c) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set 
(do not split the data into train and test, as we are not performing supervised learning in this
exercise)

Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.

In each cluster, determine which family is the majority by reading the true labels.
Repeat for genus and species.

Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss between the true labels and the labels assigned by clusters.


In [9]:
# Original data without ID column (no training/test division)
labels = ['Family', 'Genus', 'Species']
dataFeatures = data.drop(columns = labels)
dataTarget   = data[labels]
 
# Perform k-means clustering 50 times
allHammingDistances = []
allHammingLosses = []
for i in range(0,50):
    print('\n------ MC Simulation #', i+1, '------')
    
    # Find best k values based on silhouette scores
    silhoutteScores = []
    for k in range(2, 51):
        dataPredicted = KMeans(n_clusters = k).fit(dataFeatures).labels_
        silhoutteScores.append(skM.silhouette_score(dataFeatures, dataPredicted))
    bestK = np.argmax(silhoutteScores)
    print('Best k is found as', str(bestK), 'with silhouette score of', np.round(np.max(silhoutteScores), 4))

    # Cluster on best K
    dataPredicted = KMeans(n_clusters = bestK).fit_predict(dataFeatures) 

    # Create a dictionary for clusters 
    clustersDict = dict()
    for k in range(bestK):
        clustersDict[k] = dict()
        for label in labels:
            clustersDict[k][label] = dict()
    for i in range(len(dataPredicted)):
        k = dataPredicted[i]
        for label in labels:
            correctLabel = dataTarget[label][i]
            clustersDict[k][label][correctLabel] = clustersDict[k][label].get(correctLabel, 0) + 1
    
    # Find the majority labels (for part b) 
    majorityLabels = dict()
    for k in clustersDict:
        majorityLabels[k] = dict()
        for label in labels:
            majority = 0
            cluster = ''

            for key in clustersDict[k][label]:
                val = clustersDict[k][label][key]
                if val > majority:
                    majority = val
                    cluster = key

            majorityLabels[k][label] = cluster

    print('\nMajority labels are found as:') 
    print(pd.DataFrame(majorityLabels))
 
    # Calculate metrics  (for part c) 
    hammingDistances = []
    hammingScores = []
    hammingLosses = []
    for i in range(len(dataTarget)):

        hammingL = [] 
        for l in majorityLabels[dataPredicted[i]].keys():
            hammingL.append(skM.hamming_loss([dataTarget[labels].iloc[i][l]], [majorityLabels[dataPredicted[i]][l]]))

        hammingLosses.append(np.mean(hammingL))
        hammingDistances.append(np.sum(hammingL))
        hammingScores.append(1 - np.mean(hammingL))
 
    print('\nMetrics:') 
    print('Average Hamming distance:', np.round(np.mean(hammingDistances), 4))
    print('Average Hamming score:', np.round(1 - np.mean(hammingLosses), 4))
    print('Average Hamming loss:', np.round(np.mean(hammingLosses), 4))

    # Store the average values for each MC simulation
    allHammingDistances.append(np.mean(hammingDistances))
    allHammingLosses.append(np.mean(hammingLosses)) 



------ MC Simulation # 1 ------
Best k is found as 2 with silhouette score of 0.3788

Majority labels are found as:
                         0                       1
Family             Hylidae         Leptodactylidae
Genus            Hypsiboas               Adenomera
Species  HypsiboasCordobae  AdenomeraHylaedactylus

Metrics:
Average Hamming distance: 0.8956
Average Hamming score: 0.7015
Average Hamming loss: 0.2985

------ MC Simulation # 2 ------
Best k is found as 2 with silhouette score of 0.3853

Majority labels are found as:
                              0                  1
Family          Leptodactylidae            Hylidae
Genus                 Adenomera          Hypsiboas
Species  AdenomeraHylaedactylus  HypsiboasCordobae

Metrics:
Average Hamming distance: 0.8956
Average Hamming score: 0.7015
Average Hamming loss: 0.2985

------ MC Simulation # 3 ------
Best k is found as 2 with silhouette score of 0.3788

Majority labels are found as:
                              0      

In [10]:
print('\nAverage metrics for all MC iterations:') 
print('Hamming distance:', np.round(np.mean(allHammingDistances), 4))
print('Hamming score:', np.round(1 - np.mean(allHammingLosses), 4))
print('Hamming loss:', np.round(np.mean(allHammingLosses), 4))


Average metrics for all MC iterations:
Hamming distance: 0.8956
Hamming score: 0.7015
Hamming loss: 0.2985


## 3. ISLR 12.6.2

![Answer ISLR 12.6.2](ISLR_12-6-2.jpg)